## ============================================================================
## CONFIDENCE-BASED PSEUDO-LABELING FOR BCCC DARKNET DATASET
## Ensemble Approach: Isolation Forest + Autoencoder
## ============================================================================


## ============================================================================
## CONFIGURATION
## ============================================================================


In [4]:
# ==========================
# Imports
# ==========================
import os

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

import joblib
import warnings
warnings.filterwarnings("ignore")

# ==========================
# Configuration
# ==========================

# TODO: CHANGE THIS TO YOUR ACTUAL CSV PATH
DATA_PATH = r"bccc_darknet.csv"

OUTPUT_DIR = "./pseudo_labeling_artifacts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

RANDOM_STATE = 42
CONTAMINATION = 0.05          # expected anomaly ratio for IF (5%) [tunable]
AUTOENCODER_EPOCHS = 50
AUTOENCODER_BATCH_SIZE = 256
TOP_N_FEATURES = 50           # top-k variance features for stability

print("Configuration:")
print(f"  DATA_PATH     = {DATA_PATH}")
print(f"  OUTPUT_DIR    = {OUTPUT_DIR}")
print(f"  CONTAMINATION = {CONTAMINATION}")


Configuration:
  DATA_PATH     = bccc_darknet.csv
  OUTPUT_DIR    = ./pseudo_labeling_artifacts
  CONTAMINATION = 0.05


## ============================================================================
## Data loading & preprocessing
### Cell 2 – Load data and basic cleaning (code)
## ============================================================================



In [5]:
print("=" * 70)
print("STEP 1: LOAD & CLEAN DATA")
print("=" * 70)

# Load CSV
data = pd.read_csv(DATA_PATH)
print(f"✓ Dataset loaded: {data.shape}")
print(f"  Columns (first 10): {data.columns[:10].tolist()}")

# Select numeric features only (drop non-numeric for this pipeline)
features = data.select_dtypes(include=["float64", "int64"]).copy()
print(f"\n✓ Numeric features selected: {features.shape[1]} features")

# Replace inf/-inf with NaN
features.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaN with median of each column
features.fillna(features.median(), inplace=True)

# Clip extreme values to avoid numerical issues
features = features.clip(-1e9, 1e9)

print("✓ Cleaning done:")
print(f"  Final numeric feature matrix: {features.shape}")


STEP 1: LOAD & CLEAN DATA
✓ Dataset loaded: (25538, 475)
  Columns (first 10): ['flow_id', 'timestamp', 'src_ip', 'src_port', 'dst_ip', 'dst_port', 'protocol', 'duration', 'packets_count', 'fwd_packets_count']

✓ Numeric features selected: 467 features
✓ Cleaning done:
  Final numeric feature matrix: (25538, 467)


## ============================================================================
## Cell 3 – Feature selection (variance-based) and scaling (code)
## ============================================================================

In [6]:
print("\n" + "=" * 70)
print("STEP 2: FEATURE SELECTION & SCALING")
print("=" * 70)

# --- Feature selection: top-N by variance (unsupervised) ---
feature_variance = features.var().sort_values(ascending=False)

if len(features.columns) > TOP_N_FEATURES:
    selected_features = feature_variance.nlargest(TOP_N_FEATURES).index
    features_sel = features[selected_features].copy()
    print(f"✓ Selected top {TOP_N_FEATURES} features by variance")
else:
    features_sel = features.copy()
    print(f"✓ Using all {len(features.columns)} features (less than TOP_N_FEATURES)")

print(f"  Selected feature matrix: {features_sel.shape}")

# Save selected feature names for downstream reference
feature_names = features_sel.columns.tolist()

# --- Standardization ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features_sel)

print("\n✓ Data scaled with StandardScaler")
print(f"  Shape: {X_scaled.shape}")
print(f"  Mean (overall): {X_scaled.mean():.6f}")
print(f"  Std  (overall): {X_scaled.std():.6f}")

# Persist scaler
scaler_path = os.path.join(OUTPUT_DIR, "scaler_pseudo_labeling.pkl")
joblib.dump(scaler, scaler_path)
print(f"✓ Scaler saved to: {scaler_path}")



STEP 2: FEATURE SELECTION & SCALING
✓ Selected top 50 features by variance
  Selected feature matrix: (25538, 50)

✓ Data scaled with StandardScaler
  Shape: (25538, 50)
  Mean (overall): 0.000000
  Std  (overall): 1.000000
✓ Scaler saved to: ./pseudo_labeling_artifacts/scaler_pseudo_labeling.pkl


## ============================================================================
## Isolation Forest model
#### Cell 4 – Train Isolation Forest & get anomaly scores (code)
## ============================================================================

In [7]:
print("\n" + "=" * 70)
print("STEP 3: ISOLATION FOREST ANOMALY DETECTION")
print("=" * 70)

iso_model = IsolationForest(
    n_estimators=200,
    contamination=CONTAMINATION,  # approximate expected outlier proportion
    random_state=RANDOM_STATE,
    n_jobs=-1
)

print("✓ Training Isolation Forest...")
iso_model.fit(X_scaled)

# Predictions: -1 = anomaly, 1 = normal
iso_preds = iso_model.predict(X_scaled)
iso_scores = iso_model.score_samples(X_scaled)  # lower = more anomalous

iso_anomalies = np.where(iso_preds == -1)[0]

print("\n✓ Isolation Forest results:")
print(f"  Total samples:         {len(X_scaled)}")
print(f"  Anomalies detected:    {len(iso_anomalies)}"
      f" ({len(iso_anomalies)/len(X_scaled)*100:.2f}%)")
print(f"  Score range:           [{iso_scores.min():.4f}, {iso_scores.max():.4f}]")

# Save model
iso_path = os.path.join(OUTPUT_DIR, "isolation_forest_model.pkl")
joblib.dump(iso_model, iso_path)
print(f"✓ Isolation Forest model saved to: {iso_path}")



STEP 3: ISOLATION FOREST ANOMALY DETECTION
✓ Training Isolation Forest...

✓ Isolation Forest results:
  Total samples:         25538
  Anomalies detected:    1243 (4.87%)
  Score range:           [-0.7684, -0.3399]
✓ Isolation Forest model saved to: ./pseudo_labeling_artifacts/isolation_forest_model.pkl


## ============================================================================
## Autoencoder model
## ============================================================================

In [8]:
print("\n" + "=" * 70)
print("STEP 4: AUTOENCODER TRAINING")
print("=" * 70)

input_dim = X_scaled.shape[1]
encoding_dim = 16  # bottleneck size (tunable)

print(f"Input dimension:   {input_dim}")
print(f"Encoding dimension:{encoding_dim}")

# Define Autoencoder architecture
input_layer = Input(shape=(input_dim,))
x = Dense(32, activation="relu")(input_layer)
encoded = Dense(encoding_dim, activation="relu")(x)

x = Dense(32, activation="relu")(encoded)
output_layer = Dense(input_dim, activation="linear")(x)

autoencoder = Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer=Adam(learning_rate=1e-3), loss="mse")

print("\n✓ Training Autoencoder (unsupervised)...")
history = autoencoder.fit(
    X_scaled, X_scaled,
    epochs=AUTOENCODER_EPOCHS,
    batch_size=AUTOENCODER_BATCH_SIZE,
    validation_split=0.1,
    shuffle=True,
    verbose=1
)

print("\n✓ Autoencoder training complete")
print(f"  Final train loss: {history.history['loss'][-1]:.6f}")
print(f"  Final val   loss: {history.history['val_loss'][-1]:.6f}")



STEP 4: AUTOENCODER TRAINING
Input dimension:   50
Encoding dimension:16


2026-01-18 08:58:51.332130: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)



✓ Training Autoencoder (unsupervised)...
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.9523 - val_loss: 0.3829
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5747 - val_loss: 0.2564
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4001 - val_loss: 0.2282
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3150 - val_loss: 0.2068
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2597 - val_loss: 0.1901
Epoch 6/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2175 - val_loss: 0.1700
Epoch 7/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1863 - val_loss: 0.1580
Epoch 8/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1580 - val_loss: 0.1514
Epoch 9/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1367 - val_loss: 0.1409
Epoch 10/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1186 - val_loss: 0.1333
Epoch 11/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1026 - val_loss: 0.1261
Epoch 12/50
90/90 ━━━━━━━━━━━━━━

## ============================================================================
## Compute reconstruction errors and AE anomalies (code)
## ============================================================================

In [9]:
print("\n" + "=" * 70)
print("STEP 5: AUTOENCODER ANOMALY SCORING")
print("=" * 70)

# Reconstruction for all samples
reconstructions = autoencoder.predict(X_scaled, verbose=0)
mse = np.mean(np.square(X_scaled - reconstructions), axis=1)

# Threshold: 95th percentile of reconstruction error (common practice) [tunable]
threshold = np.percentile(mse, 95)
ae_anomalies = np.where(mse > threshold)[0]

print("✓ Autoencoder anomaly results:")
print(f"  Threshold (95th percentile): {threshold:.6f}")
print(f"  Anomalies detected:          {len(ae_anomalies)}"
      f" ({len(ae_anomalies)/len(X_scaled)*100:.2f}%)")
print(f"  MSE range:                   [{mse.min():.6f}, {mse.max():.6f}]")

# Save AE model and errors
ae_model_path = os.path.join(OUTPUT_DIR, "autoencoder_model.h5")
err_path = os.path.join(OUTPUT_DIR, "reconstruction_errors.npy")

autoencoder.save(ae_model_path)
np.save(err_path, mse)

print(f"✓ Autoencoder model saved to: {ae_model_path}")
print(f"✓ Reconstruction errors saved to: {err_path}")



STEP 5: AUTOENCODER ANOMALY SCORING


✓ Autoencoder anomaly results:
  Threshold (95th percentile): 0.029062
  Anomalies detected:          1277 (5.00%)
  MSE range:                   [0.000193, 32.712534]
✓ Autoencoder model saved to: ./pseudo_labeling_artifacts/autoencoder_model.h5
✓ Reconstruction errors saved to: ./pseudo_labeling_artifacts/reconstruction_errors.npy


## ============================================================================
## Ensemble pseudo-labeling & confidence scoring
### Cell 7 – Ensemble logic: IF + AE → pseudo-labels + confidence (code)
## ============================================================================

In [10]:
print("\n" + "=" * 70)
print("STEP 6: ENSEMBLE PSEUDO-LABELING (IF + AE)")
print("=" * 70)

# ---- 1) Identify overlap and disagreements ----
high_confidence_anomalies = np.intersect1d(iso_anomalies, ae_anomalies)
if_only = np.setdiff1d(iso_anomalies, ae_anomalies)
ae_only = np.setdiff1d(ae_anomalies, iso_anomalies)
union_anomalies = np.union1d(iso_anomalies, ae_anomalies)

print("✓ Anomaly breakdown:")
print(f"  High confidence (IF ∩ AE): {len(high_confidence_anomalies)}")
print(f"  IF only:                    {len(if_only)}")
print(f"  AE only:                    {len(ae_only)}")
print(f"  Total unique anomalies:     {len(union_anomalies)}")
print(f"  Normal (neither):           {len(X_scaled) - len(union_anomalies)}")

# ---- 2) Initialize labels and confidence ----
# pseudo_label: 0 = normal, 1 = anomaly
pseudo_labels = np.zeros(len(X_scaled), dtype=int)
confidence_scores = np.zeros(len(X_scaled), dtype=float)

# Strategy (conservative):
#   - Label as anomaly ONLY when both models agree
#   - Give lower confidence scores to model-specific anomalies for analysis

# HIGH CONFIDENCE: IF and AE both say anomaly
pseudo_labels[high_confidence_anomalies] = 1
confidence_scores[high_confidence_anomalies] = 1.0

# MEDIUM CONFIDENCE: only one model flags as anomaly
# (we keep label = 0 but record that they are suspicious via confidence)
confidence_scores[if_only] = 0.3
confidence_scores[ae_only] = 0.3

print("\n✓ Pseudo-labels and confidence scores created:")
print(f"  Anomalies (label=1): {np.sum(pseudo_labels == 1)}")
print(f"  Normals  (label=0):  {np.sum(pseudo_labels == 0)}")
print(f"  Confidence - min:    {confidence_scores.min():.4f}")
print(f"                max:    {confidence_scores.max():.4f}")
print(f"                mean:   {confidence_scores.mean():.4f}")



STEP 6: ENSEMBLE PSEUDO-LABELING (IF + AE)
✓ Anomaly breakdown:
  High confidence (IF ∩ AE): 317
  IF only:                    926
  AE only:                    960
  Total unique anomalies:     2203
  Normal (neither):           23335

✓ Pseudo-labels and confidence scores created:
  Anomalies (label=1): 317
  Normals  (label=0):  25221
  Confidence - min:    0.0000
                max:    1.0000
                mean:   0.0346


## Build labeled DataFrame & save (code)

In [11]:
print("\n" + "=" * 70)
print("STEP 7: BUILD & SAVE LABELED DATASET")
print("=" * 70)

# DataFrame with selected (scaled) features
df_labeled = pd.DataFrame(X_scaled, columns=feature_names)

# Add labels and meta-columns
df_labeled["pseudo_label"] = pseudo_labels
df_labeled["confidence_score"] = confidence_scores
df_labeled["if_anomaly"] = 0
df_labeled["ae_anomaly"] = 0

df_labeled.loc[iso_anomalies, "if_anomaly"] = 1
df_labeled.loc[ae_anomalies, "ae_anomaly"] = 1

print("✓ Labeled DataFrame summary:")
print(f"  Shape:   {df_labeled.shape}")
print(f"  Columns: {df_labeled.columns.tolist()}")

# Save full labeled dataset
labeled_all_path = os.path.join(OUTPUT_DIR, "bccc_labeled_all_confidence.csv")
df_labeled.to_csv(labeled_all_path, index=False)
print(f"\n✓ Saved full labeled dataset to: {labeled_all_path}")

# Optionally, create a "high-confidence" subset:
# all high-confidence anomalies + all normals
high_conf_mask = (pseudo_labels == 1)
df_high_conf = pd.concat(
    [df_labeled[high_conf_mask], df_labeled[pseudo_labels == 0]],
    axis=0
).reset_index(drop=True)

labeled_high_conf_path = os.path.join(OUTPUT_DIR, "bccc_labeled_high_confidence.csv")
df_high_conf.to_csv(labeled_high_conf_path, index=False)
print(f"✓ Saved high-confidence dataset to: {labeled_high_conf_path}")



STEP 7: BUILD & SAVE LABELED DATASET
✓ Labeled DataFrame summary:
  Shape:   (25538, 54)
  Columns: ['packet_IAT_min', 'packets_IAT_mode', 'packets_IAT_median', 'packets_IAT_mean', 'packet_IAT_max', 'packet_IAT_total', 'fwd_packets_IAT_min', 'fwd_packets_IAT_mode', 'fwd_packets_IAT_median', 'fwd_packets_IAT_mean', 'fwd_packets_IAT_max', 'fwd_packets_IAT_total', 'bwd_packets_IAT_min', 'bwd_packets_IAT_mode', 'bwd_packets_IAT_median', 'bwd_packets_IAT_mean', 'bwd_packets_IAT_max', 'bwd_packets_IAT_total', 'avg_bwd_bulk_rate', 'bytes_rate', 'avg_fwd_bulk_rate', 'variance_fwd_packets_delta_len', 'variance_fwd_payload_bytes_delta_len', 'variance_payload_bytes_delta_len', 'variance_packets_delta_len', 'fwd_bytes_rate', 'bwd_bytes_rate', 'fwd_payload_bytes_variance', 'fwd_segment_size_variance', 'total_payload_bytes', 'payload_bytes_variance', 'segment_size_variance', 'bwd_total_payload_bytes', 'bwd_bulk_total_size', 'fwd_total_payload_bytes', 'fwd_bulk_total_size', 'variance_bwd_payload_byt

## Text summary (code)

In [12]:
num_total = len(X_scaled)
num_anom = int(np.sum(pseudo_labels == 1))
num_norm = int(np.sum(pseudo_labels == 0))

print("\n" + "=" * 70)
print("SUMMARY")
print("=" * 70)
print(f"Total samples:               {num_total}")
print(f"Anomalies (pseudo_label=1):  {num_anom} ({num_anom/num_total*100:.2f}%)")
print(f"Normals   (pseudo_label=0):  {num_norm} ({num_norm/num_total*100:.2f}%)")
print(f"High-confidence anomalies:   {len(high_confidence_anomalies)}")
print(f"IF-only anomalies:           {len(if_only)}")
print(f"AE-only anomalies:           {len(ae_only)}")
print(f"Confidence mean:             {confidence_scores.mean():.4f}")

print("\nNext step: Use "
      "'bccc_labeled_high_confidence.csv' to train a supervised model (e.g., Random Forest).")



SUMMARY
Total samples:               25538
Anomalies (pseudo_label=1):  317 (1.24%)
Normals   (pseudo_label=0):  25221 (98.76%)
High-confidence anomalies:   317
IF-only anomalies:           926
AE-only anomalies:           960
Confidence mean:             0.0346

Next step: Use 'bccc_labeled_high_confidence.csv' to train a supervised model (e.g., Random Forest).
